# Consolidação de Base de Dados - IEEE Xplore
Este notebook tem como objetivo unificar as exportações de metadados da base IEEE Xplore, padronizando os dados para a etapa de extração da revisão da literatura (Metodologia TEMAC).

In [6]:
import os
import pandas as pd

# Definição do diretório onde estão os arquivos brutos exportados
pasta_origem = "/home/silvio/Projetos/TEMAC/IEEE_XPRES"

### 1. Mapeamento dos Arquivos CSV
Verificamos o diretório e listamos todos os arquivos com a extensão `.csv` para garantir que apenas os dados corretos sejam processados.

In [7]:
if not os.path.exists(pasta_origem):
    print(f"Erro: A pasta '{pasta_origem}' não foi encontrada no diretório atual.")
    arquivos_csv = []
else:
    # Lista apenas os arquivos terminados em .csv usando list comprehension
    arquivos_csv = [arquivo for arquivo in os.listdir(pasta_origem) if arquivo.endswith('.csv')]
    
    if not arquivos_csv:
        print(f"Nenhum arquivo CSV encontrado na pasta '{pasta_origem}'.")
    else:
        print(f"Encontrados {len(arquivos_csv)} arquivos CSV prontos para leitura.")

Encontrados 17 arquivos CSV prontos para leitura.


### 2. Leitura e Concatenação
Aqui iteramos sobre a lista de arquivos mapeados, carregamos cada um em um DataFrame do Pandas e, em seguida, unimos todos em uma única estrutura. O `pd.concat` se encarrega de alinhar as colunas e manter o cabeçalho apenas uma vez.

In [8]:
lista_dataframes = []

if arquivos_csv:
    for nome_arquivo in arquivos_csv:
        caminho_completo = os.path.join(pasta_origem, nome_arquivo)
        
        # Leitura do CSV
        df = pd.read_csv(caminho_completo, encoding='utf-8') 
        lista_dataframes.append(df)
        print(f"Lido: {nome_arquivo} - {len(df)} linhas")

    # Junta tudo em uma única base de dados
    base_consolidada = pd.concat(lista_dataframes, ignore_index=True)
    total_inicial = len(base_consolidada)
    print(f"\nTotal de registros empilhados: {total_inicial}")

Lido: export2026.02.20-18.39.57.csv - 1 linhas
Lido: export2026.02.20-18.41.22.csv - 13 linhas
Lido: export2026.02.20-18.39.10.csv - 2 linhas
Lido: export2026.02.20-18.48.20.csv - 50 linhas
Lido: export2026.02.20-18.44.57.csv - 982 linhas
Lido: export2026.02.20-18.39.39.csv - 21 linhas
Lido: export2026.02.20-18.48.55.csv - 14 linhas
Lido: export2026.02.20-18.46.06.csv - 60 linhas
Lido: export2026.02.20-18.41.43.csv - 53 linhas
Lido: export2026.02.20-18.47.06.csv - 588 linhas
Lido: export2026.02.20-18.40.13.csv - 30 linhas
Lido: export2026.02.20-18.46.33.csv - 81 linhas
Lido: export2026.02.20-18.40.51.csv - 1 linhas
Lido: export2026.02.20-18.47.55.csv - 51 linhas
Lido: export2026.02.20-18.40.37.csv - 1 linhas
Lido: export2026.02.20-18.42.07.csv - 14 linhas
Lido: export2026.02.20-18.47.30.csv - 35 linhas

Total de registros empilhados: 1997


### 3. Limpeza de Duplicatas e Exportação
Para garantir a integridade do portfólio bibliográfico final, removemos publicações duplicadas utilizando o Título do Documento e o DOI como chave de verificação. Por fim, exportamos a base limpa.

In [9]:
if arquivos_csv and 'base_consolidada' in locals():
    # Remoção de duplicatas baseada em Título e DOI
    base_deduplicada = base_consolidada.drop_duplicates(subset=['Document Title', 'DOI'], keep='first')
    total_final = len(base_deduplicada)
    
    print("--- Resumo da Limpeza ---")
    print(f"Registros originais (com possíveis duplicatas): {total_inicial}")
    print(f"Registros únicos mantidos: {total_final}")
    print(f"Duplicatas removidas: {total_inicial - total_final}")
    
    # Salva o arquivo final consolidado
    arquivo_saida = "base_ieee_consolidada.csv"
    base_deduplicada.to_csv(arquivo_saida, index=False, encoding='utf-8')
    print(f"\nBase final pronta e salva como: {arquivo_saida}")

--- Resumo da Limpeza ---
Registros originais (com possíveis duplicatas): 1997
Registros únicos mantidos: 1646
Duplicatas removidas: 351

Base final pronta e salva como: base_ieee_consolidada.csv


In [11]:
df_ieee = pd.read_csv("base_ieee_consolidada.csv")

# 2. Dicionário base de tradução AMPLIADO (aproveitando mais dados do IEEE)
de_para_scopus = {
    "Document Title": "Title",
    "Authors": "Authors",
    "Author Affiliations": "Affiliations",
    "Publication Title": "Source title",
    "Publication Year": "Year",
    "Abstract": "Abstract",
    "DOI": "DOI",
    "Article Citation Count": "Cited by",
    "Author Keywords": "Author Keywords",
    "IEEE Terms": "Index Keywords",
    # NOVAS COLUNAS MAPEADAS PARA ENRIQUECER A BASE:
    "Volume": "Volume",
    "Issue": "Issue",
    "Start Page": "Page start",
    "End Page": "Page end",
    "Publisher": "Publisher",
    "ISSN": "ISSN"
}

df_ieee_scopus_format = df_ieee.rename(columns=de_para_scopus)

# 3. Adicionar as colunas obrigatórias de sistema da Scopus e a de Referências vazia
df_ieee_scopus_format["Source"] = "Scopus"
df_ieee_scopus_format["Document Type"] = "Article"
df_ieee_scopus_format["EID"] = "IEEE_" + df_ieee_scopus_format.index.astype(str)
df_ieee_scopus_format["References"] = "NA" # Previne erros de incompatibilidade no merge

# 4. Manter apenas as colunas mapeadas e as novas
colunas_scopus = list(de_para_scopus.values()) + ["Source", "Document Type", "EID", "References"]
df_ieee_final = df_ieee_scopus_format[[col for col in colunas_scopus if col in df_ieee_scopus_format.columns]].copy()

# 5. O ATALHO NINJA (Crucial para o R não travar nas afiliações da IEEE)
# Troca ponto-e-vírgula por vírgula nas afiliações para evitar o loop infinito no bibliometrix
if "Affiliations" in df_ieee_final.columns:
    df_ieee_final["Affiliations"] = df_ieee_final["Affiliations"].astype(str).str.replace(";", ",", regex=False)

# 6. Blindagem de campos vazios para o R não engasgar
df_ieee_final = df_ieee_final.astype(object).fillna("")

# 7. Salvar o arquivo
arquivo_adaptado = "base_ieee_adaptada_scopus.csv"
df_ieee_final.to_csv(arquivo_adaptado, index=False, encoding='utf-8')

print(f"Base da IEEE harmonizada (Enriquecida e Blindada) salva como: {arquivo_adaptado}")

Base da IEEE harmonizada (Enriquecida e Blindada) salva como: base_ieee_adaptada_scopus.csv
